In [86]:
import json
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
import matplotlib.dates as mdates

In [87]:
collection_slug="pudgypenguins" # [pudgypenguins,chimpersnft,doodles-official]
collection_event_pth = f'./data/{collection_slug}/collection_event_data.csv'
collection_event= pd.read_csv(collection_event_pth)

# collection_event_stats_trade=collection_event[collection_event['has_trades']==True]
collection_event['event_timestamp_iso'] = pd.to_datetime(
    collection_event['event_timestamp_iso'].str.replace('Z', ''),
    utc=True
)

In [88]:
collection_event.head()

,collection_slug,identifier,event_type,order_hash,chain,protocol_address,closing_date,closing_date_iso,quantity,seller,buyer,payment_quantity,payment_token,payment_decimals,payment_symbol,payment_symbol_amount,transaction_hash,event_timestamp,event_timestamp_iso
0,pudgypenguins,0,sale,NaN,ethereum,NaN,1732036547,2024-11-19T17:15:47+00:00Z,1,0x3784f3d7cf80309c240ee8b78de8d32b7a9163fd,0x1fbb89924aa838622593358471c2cff33f6a985c,18000000000000000000,0x0000000000000000000000000000000000000000,18,ETH,18.0000,0xb6776baaf773f765a87295104851a88814a7d998c754...,1732036547,2024-11-19 17:15:47+00:00
1,pudgypenguins,1,sale,0x13ab2ff99d14c12d628a70eb895d647afc6c3a67c3e4...,ethereum,0x00000000006c3852cbef3e08e8df289169ede581,1673049515,2023-01-06T23:58:35+00:00Z,1,0xc80e301a685016fbce6c984022150e33b2e8f5f1,0xcce98763ff5a9ff5baf8b15abc456077a1e84f2a,9990000000000000000,0x0000000000000000000000000000000000000000,18,ETH,9.9900,0x49222e91995002e6d13d734c2590fc607dacf7f7f64a...,1673049515,2023-01-06 23:58:35+00:00
2,pudgypenguins,1,sale,0xf9f0a71cd7844407d4183783b9376931bf829bd4c465...,ethereum,0x00000000006c3852cbef3e08e8df289169ede581,1673047727,2023-01-06T23:28:47+00:00Z,1,0xba169584e7c13efd57157657b2efdff918be08c7,0xcd7bd7f7eb19ffc8ee505e8eaa7b78100182d395,7508700000000000000,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,18,WETH,7.5087,0xf61f9e500d5098451cf2025ce309228dde194796fc1b...,1673047727,2023-01-06 23:28:47+00:00
3,pudgypenguins,1,sale,0xac399619ee5953fce52922a91b2cb04265ea59907241...,ethereum,0x7f268357a8c2552623316e2562d90e642bb538e5,1650515500,2022-04-21T04:31:40+00:00Z,1,0x4d53f9106edd415ea63c847c9aa8d12b31880e05,0x2cbc202392c0f0c846bf028777a5e9b4e49d9fac,6580000000000000000,0x0000000000000000000000000000000000000000,18,ETH,6.5800,0x18faed50a64853c2544ef09153f9b9214cbca77702e1...,1650515500,2022-04-21 04:31:40+00:00
4,pudgypenguins,1,sale,NaN,ethereum,NaN,1649044315,2022-04-04T03:51:55+00:00Z,1,0xfa8e8b984bf77c1f0bea1c6837c439c7ef080554,0x4d53f9106edd415ea63c847c9aa8d12b31880e05,4269000000000000000,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,18,WETH,4.2690,0x048ea16979944fa731baac826b06e48b45d3a54319fe...,1649044315,2022-04-04 03:51:55+00:00


In [ ]:
daily_floor = collection_event.groupby(
    pd.Grouper(key='event_timestamp_iso', freq='D')
)['payment_symbol_amount'].min().reset_index()
daily_floor.columns = ['date', 'floor_price_eth']

daily_floor['floor_7d_avg'] = daily_floor['floor_price_eth'].rolling(7, min_periods=1).mean()

In [ ]:
daily_volume = collection_event.groupby(
    pd.Grouper(key='event_timestamp_iso', freq='D')
).agg(
    total_volume_eth=('payment_symbol_amount', 'sum'),
    trade_count=('event_type', 'count')
).reset_index()
daily_volume.columns = ['date', 'total_volume','trade_count']

daily_volume['volume_7d_avg'] = daily_volume['total_volume'].rolling(7, min_periods=1).mean()
daily_volume['trade_7d_avg'] = daily_volume['trade_count'].rolling(7, min_periods=1).mean()

In [84]:
# merge daily_floor and daily_volume by date
collection_event_date= pd.merge(
    daily_floor,
    daily_volume,
    on='date',
    how='left'
)
collection_event_date.to_csv(f'./data/{collection_slug}/collection_event_data_date.csv', index=False)

In [85]:
collection_event_date

,date,floor_price_eth,floor_7d_avg,total_volume,trade_count,volume_7d_avg,trade_7d_avg
0,2021-10-17 00:00:00+00:00,0.1430,0.143000,1055.839108,839,1055.839108,839.000000
1,2021-10-18 00:00:00+00:00,0.0990,0.121000,6292.084141,4177,3673.961624,2508.000000
2,2021-10-19 00:00:00+00:00,0.0810,0.107667,801.317179,653,2716.413476,1889.666667
3,2021-10-20 00:00:00+00:00,0.7178,0.260200,682.492690,581,2207.933279,1562.500000
4,2021-10-21 00:00:00+00:00,0.7000,0.348160,359.600759,307,1838.266775,1311.400000
...,...,...,...,...,...,...,...
1297,2025-05-06 00:00:00+00:00,3.0300,3.174285,421.700857,120,176.408074,51.000000
1298,2025-05-07 00:00:00+00:00,3.1700,3.222857,786.970415,221,283.886053,80.857143
1299,2025-05-08 00:00:00+00:00,3.0500,3.248571,764.105895,231,370.813286,106.428571
1300,2025-05-09 00:00:00+00:00,0.9300,2.940000,1619.797367,1123,575.552657,259.000000
